In [1]:
import numpy as np
import json
import os
import re
import collections
from sklearn.datasets.base import load_files
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import codecs
import pickle
from gensim.models.word2vec import LineSentence
from gensim.models.fasttext import FastText
import sys
from __future__ import division
from collections import OrderedDict
import pandas as pd
from sklearn.datasets import load_files
from collections import OrderedDict
import sys

In [2]:
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms.matrix_factorization import SVD
from surprise.model_selection import KFold
from surprise import accuracy

In [3]:
def normalize_name(name):
    name = name.lower()
    name = re.sub(r"[.,\/#!$%\^\*;:{}=\_`~()@]", ' ', name)
    name = re.sub(r'\s+', ' ', name).strip()
    return name

In [4]:
#Load Training Data change path name as per your directory 
path = '/home/irlab/Documents/recsys/mpd/data'
filenames = os.listdir(path)
mapping = {}
playlist_prob = OrderedDict()

In [5]:
#Train FastText Model for finding similarity
#data.txt is created from the playlist names
#change location as per your 'files' folder location
def train_playlist_embedding(filenames):    
    #Change this as per you files folder location
    data = LineSentence('/home/irlab/files/data.txt')
    model = FastText(size=100)
    print('Building Vocab....')
    model.build_vocab(data)
    print('Training....')
    model.train(data, total_examples=model.corpus_count, epochs=model.iter)
    print(model)
    return model

In [6]:
model = train_playlist_embedding(filenames)

Building Vocab....
Training....


/home/irlab/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  


FastText(vocab=9132, size=100, alpha=0.025)


In [7]:
#Find Similar Playlists with given name
def find_similar(playlist_name,model = model):
    names = []
    similar_playlists = model.wv.similar_by_word(playlist_name,20)
    similar_playlists = dict(similar_playlists)
    sim_playlist_name = [key for (key,value) in sorted(similar_playlists.items(),key=lambda x:x[1],reverse=True)]
    return sim_playlist_name

In [8]:
#Mapping of playlist name to filename
def load_filemapping():
    #Change this as per you files folder location
    fp = open('/home/irlab/files/mappings/track_filename.json')
    mpd_mapping = json.load(fp)
    return mpd_mapping

In [9]:
mpd_mapping = load_filemapping()

In [10]:
#Change this as per you files folder location
fp = open('/home/irlab/files/stats/album_stats.json')
album_map = json.load(fp)
album_stat = sum(album_map.values())

#Change this as per you files folder location
fp = open('/home/irlab/files/stats/artist_stats.json')
artist_map = json.load(fp)
artist_stat = sum(artist_map.values())

#Change this as per you files folder location
fp = open('/home/irlab/files/stats/tracks_stats.json')
track_map = json.load(fp)
track_stat = sum(track_map.values())

print(album_stat,artist_stat,track_stat)

(66346428, 66346428, 66346428)


In [11]:
#Generate weights for the tracks
def create_weighted_playlists(playlist_name,playlist_prob):
    try:
        similar_playlists = find_similar(playlist_name)
    except KeyError:
        temp = playlist_name.split()
        playlist_name = ''.join(temp)
        similar_playlists = find_similar(playlist_name)
        
    #print(similar_playlists)
    for similar_playlist in similar_playlists:
        if similar_playlist in mpd_mapping.keys():    
            #print(similar_playlist)
            name = mpd_mapping[similar_playlist]
            filename = os.sep.join((path,name))
            fp = open(filename,'r')
            playlist_file = json.load(fp)
            mpd_data = playlist_file['playlists']
            playlist_prob[similar_playlist] = {}
            for playlist in mpd_data:
                if normalize_name(playlist['name']) in similar_playlist:
                    for track in playlist['tracks']:
                        p1 = artist_map[track['artist_uri']]/artist_stat
                        p2 = album_map[track['album_uri']]/album_stat
                        p3 = track_map[track['track_uri']]/track_stat
                        playlist_prob[similar_playlist][track['track_uri']] = (p1+p2+p3)/(1+track['pos'])
    
    return playlist_prob

In [12]:
#Create Rating matrix
def create_matrix(playlist_prob):
    pd.DataFrame(data=playlist_prob,dtype=np.int64)
    df = pd.DataFrame(data=playlist_prob)
    df = df.fillna(0)
    df = df.T
    return df

In [13]:
#Load Data in surprise format
def load_data(tracks,playlists,ratings):
    ratings_dict = {'itemID': tracks,
                    'userID': playlists,
                    'rating': ratings}
    df1 = pd.DataFrame(ratings_dict)
    reader = Reader(rating_scale=(0,1))
    data = Dataset.load_from_df(df1[['userID','itemID','rating']],reader)
    #print(df1)
    return data

In [35]:
#Train using SVD and Gradient Descent
def train(data):
    kf = KFold(n_splits=10)
    algo = SVD(n_epochs=43,lr_all=0.005,reg_all=0.065)
    for trainset,testset in kf.split(data):
        algo.fit(trainset)
        prediction = algo.test(testset)
        print(accuracy.rmse(prediction,verbose=True))
    return algo

In [20]:
#Predict ratings for unknown tracks and filter them out
def test(playlist_name,tracks,algo,seed_tracks):
    tracks_pred = [n for n in tracks if n not in seed_tracks]
    Recommend = {}
    for track in tracks_pred:
        try:
            predict = algo.predict(playlist_name,track)
        except UnicodeEncodeError:
            predict = algo.predict('playlist_test',track)
            
        Recommend[track] = predict[3]
    recommend_list = [key for (key,value) in sorted(Recommend.items(),key=lambda x:x[1],reverse=True)]
    return recommend_list

In [16]:
#If we fall short of 500 tracks pick popular tracks
def random_tracks(recommend_list,seed_tracks):
    tracks = [key for (key,value) in sorted(track_map.items(),key=lambda x:x[1],reverse=True)]
    extend_tracks = [track for track in tracks if track not in recommend_list]
    extend_tracks = [track for track in extend_tracks if track not in seed_tracks]
    diff = 500 - len(recommend_list)
    print(len(extend_tracks[0:diff]))
    for name in extend_tracks[0:diff]:
        print(name)
        recommend_list.append(name)
    return recommend_list

In [39]:
#change this path as per your system
path1 = '/home/irlab/Documents/recsys/challenge'
challenge_file = 'challenge_set.json'
filename = os.sep.join((path1,challenge_file))
fp = open(filename)
challenge_set = json.load(fp)
playlists = challenge_set['playlists']
dict_playlist = {}

In [18]:
import csv

In [40]:
string = ['team_info','main','Dark_Horse_1','201711040@daiict.ac.in']

#Change this path as per your system
with open('/home/irlab/Documents/recsys/mpd/codes/output/final_submission.csv','w') as csvfile:
        filewriter = csv.writer(csvfile,delimiter = ',')
        filewriter.writerow(string)

count=0
for playlist in playlists:
    list_temp = []
    seed_tracks = []
    recommend_list = []
    if 'name' in playlist.keys() and playlist['num_samples']==0:
        pid = playlist['pid']
        list_temp.insert(0,pid)
        count+=1
        print(count,playlist['name'])
        tracks = playlist['tracks']
        playlist_prob.clear()
        playlist_prob[normalize_name(playlist['name'])] = {}
        print('Finding Similar Playlists...')
        playlist_prob = create_weighted_playlists(normalize_name(playlist['name']),playlist_prob)
        print('Creating Rating matrix...')
        df = create_matrix(playlist_prob)
        column_names = list(df.columns)
        row_names = list(df.index)
        matrix = df.values
        row,col = np.nonzero(matrix)
        row = row.tolist()
        col = col.tolist()
        n_tracks = [column_names[i] for i in col]
        playlists = [row_names[i] for i in row]
        ratings = [matrix[r][c] for (r,c) in zip(row,col)]
        print('Loading data for training')
        data = load_data(n_tracks,playlists,ratings)
        print('Training....')
        algo = train(data)
        print('Predicting....')
        recommend_list = test(normalize_name(playlist['name']),n_tracks,algo,seed_tracks)
        print('Playlist Size:',len(recommend_list))
        if len(recommend_list[0:500])<500:
            print('Picking popular tracks...')
            recommend_list = random_tracks(recommend_list,seed_tracks)
            print('New Playlist Size:',len(recommend_list))
        list_temp.extend(recommend_list[0:500])
        print('Writing to file.....')
        #Change this path as per your system
        with open('/home/irlab/Documents/recsys/mpd/codes/output/final_submission.csv','a') as csvfile1:
            filewriter = csv.writer(csvfile1,delimiter = ',')
            filewriter.writerow(list_temp)

(1, u'spanish playlist')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0067
0.006682765610642953
RMSE: 0.0039
0.0039007648944393123
RMSE: 0.0048
0.004848799014776792
RMSE: 0.0052
0.005213900661381372
RMSE: 0.0074
0.007387357943759325
RMSE: 0.0066
0.006562866770980833
RMSE: 0.0068
0.0067873324505607815
RMSE: 0.0082
0.00821463524897264
RMSE: 0.0068
0.006784499907960396
RMSE: 0.0045
0.00446673642141502
Predicting....
('Playlist Size:', 3120)
Writing to file.....
(2, u'Groovin')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0063
0.006305743764333427
RMSE: 0.0091
0.00907289826082998
RMSE: 0.0039
0.0038917905398267423
RMSE: 0.0060
0.005962013035781444
RMSE: 0.0036
0.0036024102698511335
RMSE: 0.0095
0.009467419212741721
RMSE: 0.0030
0.002995418133294174
RMSE: 0.0051
0.005101673068892756
RMSE: 0.0063
0.00629248739544407
RMSE: 0.0038
0.003835596194856322
Predicting....
('Playlist Size

Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0103
0.01030159368380318
RMSE: 0.0095
0.009507702245720895
RMSE: 0.0068
0.006770450542918358
RMSE: 0.0074
0.007441402672054855
RMSE: 0.0090
0.00896116204332249
RMSE: 0.0107
0.010688128108297217
RMSE: 0.0046
0.004632830181632318
RMSE: 0.0062
0.006205249459443299
RMSE: 0.0039
0.003894335554278186
RMSE: 0.0047
0.004674741948354138
Predicting....
('Playlist Size:', 1670)
Writing to file.....
(18, u'Upbeat')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0073
0.007325073394477934
RMSE: 0.0087
0.008737562562312916
RMSE: 0.0044
0.004359474888135076
RMSE: 0.0047
0.004658287437514229
RMSE: 0.0063
0.006313802561735757
RMSE: 0.0053
0.005302476602284111
RMSE: 0.0067
0.006740738745793951
RMSE: 0.0048
0.0048150839450346004
RMSE: 0.0044
0.004431392597293794
RMSE: 0.0072
0.007202592134663666
Predicting....
('Playlist Size:', 3828)
Writing to file.....
(19, u'Adventure')
Findi

Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0056
0.0055991477825501545
RMSE: 0.0056
0.005620100943308944
RMSE: 0.0046
0.004568081729581688
RMSE: 0.0031
0.003083565994298374
RMSE: 0.0031
0.0030723026992400286
RMSE: 0.0053
0.00531384426125024
RMSE: 0.0069
0.006921980636026842
RMSE: 0.0059
0.005946026680536428
RMSE: 0.0040
0.00400053819035603
RMSE: 0.0042
0.004197939707360868
Predicting....
('Playlist Size:', 1672)
Writing to file.....
(34, u'Banda')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0069
0.00691704809677765
RMSE: 0.0012
0.0012290848876500588
RMSE: 0.0124
0.012439586674082263
RMSE: 0.0102
0.010172541692207736
RMSE: 0.0108
0.010836154580044794
RMSE: 0.0104
0.010442209605165802
RMSE: 0.0083
0.008288184334002663
RMSE: 0.0090
0.009031684500434161
RMSE: 0.0062
0.006244275833199026
RMSE: 0.0084
0.00839336630392782
Predicting....
('Playlist Size:', 1490)
Writing to file.....
(35, u'\U0001f60d\U0001f6

Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0101
0.010133783004602
RMSE: 0.0079
0.007939442075634226
RMSE: 0.0096
0.009601771934420085
RMSE: 0.0092
0.009167348112547229
RMSE: 0.0042
0.004221022931572998
RMSE: 0.0084
0.008387562988455066
RMSE: 0.0073
0.007272768509776663
RMSE: 0.0041
0.004099743118491838
RMSE: 0.0072
0.007203251551131032
RMSE: 0.0063
0.006288159961597529
Predicting....
('Playlist Size:', 2157)
Writing to file.....
(50, u'grad party')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0033
0.0032882460872641897
RMSE: 0.0038
0.0037691318414959925
RMSE: 0.0029
0.0028869067098312583
RMSE: 0.0035
0.003525034846538548
RMSE: 0.0032
0.0032089250517126212
RMSE: 0.0035
0.0035072399174476466
RMSE: 0.0031
0.00305183868815671
RMSE: 0.0029
0.0029478781775977457
RMSE: 0.0028
0.0027743380863023874
RMSE: 0.0038
0.0037891233779080904
Predicting....
('Playlist Size:', 4932)
Writing to file.....
(51, u'Awesome 

Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0232
0.02316724058200427
RMSE: 0.0065
0.006484316608564998
RMSE: 0.0074
0.007375555516685842
RMSE: 0.0044
0.004350865965491437
RMSE: 0.0063
0.006261018406903573
RMSE: 0.0062
0.00623219538324401
RMSE: 0.0142
0.014210492529444082
RMSE: 0.0073
0.007347627263992586
RMSE: 0.0065
0.006450024716372588
RMSE: 0.0042
0.004154150937587093
Predicting....
('Playlist Size:', 1901)
Writing to file.....
(66, u'May')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0026
0.002649364110206612
RMSE: 0.0116
0.011600104654499515
RMSE: 0.0048
0.004820271390870777
RMSE: 0.0078
0.007800090752437281
RMSE: 0.0058
0.0058140222169374865
RMSE: 0.0078
0.007792365296307949
RMSE: 0.0125
0.01249688616938397
RMSE: 0.0093
0.009281020121263209
RMSE: 0.0037
0.003650774688179703
RMSE: 0.0043
0.004278241783284383
Predicting....
('Playlist Size:', 1488)
Writing to file.....
(67, u'\U0001f918\U0001f3fc\

...

Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0045
0.004493115508642799
RMSE: 0.0036
0.0035510510661379865
RMSE: 0.0064
0.006427783105218821
RMSE: 0.0062
0.0061522771377761215
RMSE: 0.0067
0.006691508414672072
RMSE: 0.0072
0.007218024881981392
RMSE: 0.0089
0.00890324202195374
RMSE: 0.0094
0.00941806252770908
RMSE: 0.0073
0.007306333628442301
RMSE: 0.0088
0.008763508145943429
Predicting....
('Playlist Size:', 1467)
Writing to file.....
(926, u'Yup')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0109
0.010922059690109447
RMSE: 0.0124
0.012442837230461268
RMSE: 0.0056
0.005638020424502849
RMSE: 0.0078
0.007841152787839221
RMSE: 0.0105
0.010531812087375803
RMSE: 0.0095
0.009527644375718989
RMSE: 0.0093
0.009317056504137056
RMSE: 0.0061
0.006103940282537717
RMSE: 0.0077
0.007712070624133405
RMSE: 0.0084
0.008366687312265026
Predicting....
('Playlist Size:', 1461)
Writing to file.....
(927, u'Love songs')
Find

Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0014
0.0014404406300639204
RMSE: 0.0063
0.00632392323153876
RMSE: 0.0016
0.0015703746544963168
RMSE: 0.0045
0.0045104265196074305
RMSE: 0.0034
0.0034445543589260246
RMSE: 0.0027
0.00273573012140985
RMSE: 0.0022
0.0022051623298024034
RMSE: 0.0030
0.002985010770803769
RMSE: 0.0056
0.0056489817788312364
RMSE: 0.0025
0.002474510466265465
Predicting....
('Playlist Size:', 1485)
Writing to file.....
(942, u'chill out')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0055
0.005534608554110825
RMSE: 0.0039
0.003936670963215796
RMSE: 0.0042
0.004205115582243045
RMSE: 0.0040
0.003973792418272289
RMSE: 0.0046
0.004596859861968529
RMSE: 0.0045
0.004479550959693347
RMSE: 0.0034
0.003400749275421135
RMSE: 0.0040
0.004022299503804755
RMSE: 0.0041
0.004071737304881403
RMSE: 0.0046
0.0046004135935096375
Predicting....
('Playlist Size:', 6312)
Writing to file.....
(943, u'rap')


Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0019
0.0019263985024584795
RMSE: 0.0088
0.008837642878707063
RMSE: 0.0013
0.0012857355289159723
RMSE: 0.0056
0.005639506150852812
RMSE: 0.0017
0.0016950949832316765
RMSE: 0.0046
0.004609659157761753
RMSE: 0.0032
0.0031918697983709956
RMSE: 0.0035
0.00345816566725844
RMSE: 0.0024
0.0023593696057687087
RMSE: 0.0046
0.00456545982521701
Predicting....
('Playlist Size:', 2275)
Writing to file.....
(958, u'dope')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0055
0.005524434244901409
RMSE: 0.0038
0.003755934784039689
RMSE: 0.0050
0.004983664584036296
RMSE: 0.0050
0.0049938054573960946
RMSE: 0.0070
0.006994014994900889
RMSE: 0.0039
0.003921513862246666
RMSE: 0.0044
0.004394608698064922
RMSE: 0.0061
0.0061067933917331625
RMSE: 0.0051
0.005121283476787247
RMSE: 0.0046
0.0046071503499745346
Predicting....
('Playlist Size:', 2633)
Writing to file.....
(959, u'Turn Up ')

Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0027
0.002698349322424378
RMSE: 0.0063
0.006259516403376287
RMSE: 0.0048
0.004814170282664357
RMSE: 0.0041
0.004052570932159679
RMSE: 0.0031
0.003112940785056676
RMSE: 0.0067
0.006709046539772547
RMSE: 0.0033
0.0033457162398990224
RMSE: 0.0060
0.006023563310705849
RMSE: 0.0056
0.005576740940411234
RMSE: 0.0048
0.004781987321392076
Predicting....
('Playlist Size:', 1219)
Writing to file.....
(973, u'Rnb')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0054
0.0054190635057445695
RMSE: 0.0067
0.006665929395712792
RMSE: 0.0053
0.0053365095400416075
RMSE: 0.0065
0.006516121192021316
RMSE: 0.0064
0.006362969837213876
RMSE: 0.0054
0.005355236773499645
RMSE: 0.0024
0.0023800903872907165
RMSE: 0.0036
0.0035758488021133345
RMSE: 0.0090
0.00896341268228465
RMSE: 0.0036
0.0035688926286167925
Predicting....
('Playlist Size:', 1979)
Writing to file.....
(974, u'korea')
Find

Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0033
0.0032621791580523044
RMSE: 0.0040
0.003990135044215311
RMSE: 0.0113
0.011302844774937221
RMSE: 0.0054
0.0053673158552831855
RMSE: 0.0034
0.0033652989650573913
RMSE: 0.0061
0.006061315217143373
RMSE: 0.0035
0.0034575097167134504
RMSE: 0.0021
0.002136058441744114
RMSE: 0.0090
0.009018683084023477
RMSE: 0.0045
0.0045360856370685555
Predicting....
('Playlist Size:', 1307)
Writing to file.....
(989, u'mixtape_3')
Finding Similar Playlists...
Creating Rating matrix...
Loading data for training
Training....
RMSE: 0.0041
0.004136265536581861
RMSE: 0.0039
0.003891589364390928
RMSE: 0.0048
0.0047821799973351556
RMSE: 0.0034
0.0033837179625887523
RMSE: 0.0048
0.004799990055048548
RMSE: 0.0040
0.003963832026821865
RMSE: 0.0133
0.013342055810329476
RMSE: 0.0037
0.0037270325032678618
RMSE: 0.0052
0.005198992544336226
RMSE: 0.0032
0.003220161740412285
Predicting....
('Playlist Size:', 2283)
Writing to file.....
(990, u"It'